## Injection benchmark

### Setup

In [1]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote ==
{__version__} {'(cython)' if is_compiled() else ''}
""")


== Python ==
3.10.4 (main, Mar 23 2022, 20:25:24) [GCC 11.3.0]

== Antidote ==
2.0.0b0 



In [2]:
cat /proc/cpuinfo | grep 'model name' | head -n 1

model name	: AMD Ryzen 7 PRO 4750U with Radeon Graphics


### Results
The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the most common use case of dependency injection, service instantiation, the overhead should be negligible.

It should be noted that in most cases the worst scenario is used, as those functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [3]:
import json
# Rough point of comparison
%timeit json.loads('{"name":"John","age":30,"city":"New York"}')

2.57 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
from antidote import world, Service, inject, Provide

class Service1(Service):
    pass

def f(s1: Service1):
    return s1

@inject
def injected_f(s1: Provide[Service1]):
    return s1

ImportError: cannot import name 'Service' from 'antidote' (/home/brabier/code/github/antidote/src/antidote/__init__.py)

In [ ]:
# Reference
s1 = world.get[Service1]() # singleton by default
%timeit f(s1)

In [ ]:
# With injection
assert injected_f() == f(s1)
%timeit injected_f()

In [ ]:
# With injection when no arguments must be provided
assert injected_f(s1) == f(s1)
%timeit injected_f(s1)

### Function call with multiple injections

In [ ]:
from antidote import Provide

class Service2(Service):
    pass

class Service3(Service):
    pass

class Service4(Service):
    pass

def f_multi(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

@inject
def injected_f_multi(s1: Provide[Service1],
                     s2: Provide[Service2],
                     s3: Provide[Service3],
                     s4: Provide[Service4]):
    return s1, s2, s3, s4

In [ ]:
# Reference
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f_multi(*args)

In [ ]:
# With injection
assert injected_f_multi() == f_multi(*args)
%timeit injected_f_multi()

In [ ]:
# With injection when no arguments must be provided
assert injected_f_multi(*args) == f_multi(*args)
%timeit injected_f_multi(*args)

### Method call
Overhead when applied on a method

In [ ]:
class Dummy:
    def method(self, s1: Service1):
        return s1
    
    def method_multi(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4
    
    @inject
    def injected_method(self, s1: Provide[Service1]):
        return s1
    
    @inject
    def injected_method_multi(self, 
                s1: Provide[Service1],
                s2: Provide[Service2],
                s3: Provide[Service3],
                s4: Provide[Service4]):
        return s1, s2, s3, s4

dummy = Dummy()

In [ ]:
# Reference
%timeit dummy.method(s1)

In [ ]:
# With injection
assert dummy.injected_method() == dummy.method(s1)
%timeit dummy.injected_method()

In [ ]:
# With injection when no arguments must be provided
assert dummy.injected_method(s1) == dummy.method(s1)
%timeit dummy.injected_method(s1)

### Method call with multiple injections

In [ ]:
class Dummy2:
    def method_multi(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4
    
    @inject
    def injected_method_multi(self, 
                s1: Provide[Service1],
                s2: Provide[Service2],
                s3: Provide[Service3],
                s4: Provide[Service4]):
        return s1, s2, s3, s4

dummy2 = Dummy2()

In [ ]:
# Reference
%timeit dummy2.method_multi(*args)

In [ ]:
# With injection
assert dummy2.injected_method_multi() == dummy2.method_multi(*args)
%timeit dummy2.injected_method_multi()

In [ ]:
# With injection when no arguments must be provided
assert dummy2.injected_method_multi(*args) == dummy2.method_multi(*args)
%timeit dummy2.injected_method_multi(*args)

### Non singleton

We compare the overhead of creating the whole service each time.

In [ ]:
class ServiceX(Service):
    __antidote__ = Service.Conf(singleton=False)

def g(s: ServiceX):
    return s

@inject
def injected_g(s: Provide[ServiceX]):
    return s

In [ ]:
# Reference
%timeit g(ServiceX())

In [ ]:
# With injection
assert isinstance(injected_g(), ServiceX)
assert injected_g() is not injected_g()
%timeit injected_g()

In [ ]:
# With injection when no arguments must be provided
%timeit injected_g(ServiceX())

### Object instantiation
Cost of creating wired objects

In [ ]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4
        
class InjectedObj:
    @inject
    def __init__(self,
                 s1: Provide[Service1],
                 s2: Provide[Service2],
                 s3: Provide[Service3],
                 s4: Provide[Service4]):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4
    

In [ ]:
# Reference
%timeit Obj(*args)

In [ ]:
# With injection
%timeit InjectedObj()

In [ ]:
# With injection when no arguments must be provided
%timeit InjectedObj(*args)

### Factory

In [ ]:
from typing import Annotated
from antidote import factory, From

class ServiceF:
    pass

@factory(singleton=False)
def service_factory() -> ServiceF:
    return ServiceF()

def h(s):
    return s

@inject
def injected_h(s: Annotated[ServiceF, From(service_factory)]):
    return s

In [ ]:
# Reference
%timeit h(service_factory())

In [ ]:
# With injection
assert isinstance(injected_h(), ServiceF)
assert injected_h() is not injected_h()
%timeit injected_h()

In [ ]:
# With injection when no arguments must be provided
%timeit injected_h(service_factory())

### Configuration


In [ ]:
from antidote import Constants, const

class Conf(Constants):
    A = const('A')
    B = const('B')

    def get(self, key):
        return key
    
def use_config(a, b):
    return a, b

@inject(dependencies=(Conf.A, Conf.B))
def injected_use_config(a, b):
    return a, b

In [ ]:
# Reference
conf = Conf()
%timeit use_config(conf.get('A'), conf.get('B'))

In [ ]:
# With injection
assert injected_use_config() == use_config(conf.get('A'), conf.get('B'))
%timeit injected_use_config()

In [ ]:
# With injection when no arguments must be provided
%timeit injected_use_config(conf.get('A'), conf.get('B'))

### Custom Provider

In [ ]:
import time
from antidote import world, inject
from antidote.core import Provider, DependencyValue
dep = object()
slow = object()

@world.provider
class SlowProvider(Provider):
    def exists(self, dependency):
        return dependency is slow
    
    def provide(self, dependency, container):
        time.sleep(.01)
        return DependencyValue("sleepy")
            

@world.provider
class CustomProvider(Provider):
    def exists(self, dependency):
        return dependency is dep
    
    def provide(self, dependency, container):
        return DependencyValue("Found it !")


@inject(dependencies=dict(d=dep))
def f_custom(d):
    return d

In [ ]:
assert f_custom() == "Found it !"
%timeit f_custom()